In [1]:
from Clasificador import ClasificadorNaiveBayes
from Datos import Datos
from EstrategiaParticionado import ValidacionCruzada, ValidacionSimple
from sklearn.naive_bayes import GaussianNB, MultinomialNB, CategoricalNB
from sklearn.model_selection import cross_val_score
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
datos1 = Datos("ConjuntosDatos/german.data")
datos2 = Datos("ConjuntosDatos/tic-tac-toe.data")

# 25% Test
cnv1 = ClasificadorNaiveBayes()
cnv2 = ClasificadorNaiveBayes()
particionado1 = ValidacionCruzada(4)
particionado2 = ValidacionSimple(25, 4)

germanErrorVC25, germanErrorCLPVC25 = cnv1.validacion(particionado1, datos1, cnv1)
tttErrorVC25, tttErrorCLPVC25 = cnv2.validacion(particionado1, datos2, cnv2)

germanErrorVS25, germanErrorCLPVS25 = cnv1.validacion(particionado2, datos1, cnv1)
tttErrorVS25, tttErrorCLPVS25 = cnv2.validacion(particionado2, datos2, cnv2)

# 20% Test
cnv1 = ClasificadorNaiveBayes()
cnv2 = ClasificadorNaiveBayes()
particionado1 = ValidacionCruzada(5)
particionado2 = ValidacionSimple(20, 5)

germanErrorVC20, germanErrorCLPVC20 = cnv1.validacion(particionado1, datos1, cnv1)
tttErrorVC20, tttErrorCLPVC20 = cnv2.validacion(particionado1, datos2, cnv2)

germanErrorVS20, germanErrorCLPVS20 = cnv1.validacion(particionado2, datos1, cnv1)
tttErrorVS20, tttErrorCLPVS20 = cnv2.validacion(particionado2, datos2, cnv2)

# 12.5% Test
cnv1 = ClasificadorNaiveBayes()
cnv2 = ClasificadorNaiveBayes()
particionado1 = ValidacionCruzada(8)
particionado2 = ValidacionSimple(12.5, 8)

germanErrorVC12_5, germanErrorCLPVC12_5 = cnv1.validacion(particionado1, datos1, cnv1)
tttErrorVC12_5, tttErrorCLPVC12_5 = cnv2.validacion(particionado1, datos2, cnv2)

germanErrorVS12_5, germanErrorCLPVS12_5 = cnv1.validacion(particionado2, datos1, cnv1)
tttErrorVS12_5, tttErrorCLPVS12_5 = cnv2.validacion(particionado2, datos2, cnv2)

# 10% Test
cnv1 = ClasificadorNaiveBayes()
cnv2 = ClasificadorNaiveBayes()
particionado1 = ValidacionCruzada(10)
particionado2 = ValidacionSimple(10, 10)

germanErrorVC10, germanErrorCLPVC10 = cnv1.validacion(particionado1, datos1, cnv1)
tttErrorVC10, tttErrorCLPVC10 = cnv2.validacion(particionado1, datos2, cnv2)

germanErrorVS10, germanErrorCLPVS10 = cnv1.validacion(particionado2, datos1, cnv1)
tttErrorVS10, tttErrorCLPVS10 = cnv2.validacion(particionado2, datos2, cnv2)

def print_test():
    print("Test 25%                      German\tTic-Tac-Toe")
    print(f"Validacion cruzada            {germanErrorVC25:2f}\t{tttErrorVC25:2f}")
    print(f"Validacion cruzada (Laplace)  {germanErrorCLPVC25:2f}\t{tttErrorCLPVC25:2f}")
    print(f"Validacion simple             {germanErrorVS25:2f}\t{tttErrorVS25:2f}")
    print(f"Validacion simple (Laplace)   {germanErrorCLPVS25:2f}\t{tttErrorCLPVS25:2f}\n")
    print("Test 20%                      German\tTic-Tac-Toe")
    print(f"Validacion cruzada            {germanErrorVC20:2f}\t{tttErrorVC20:2f}")
    print(f"Validacion cruzada (Laplace)  {germanErrorCLPVC20:2f}\t{tttErrorCLPVC20:2f}")
    print(f"Validacion simple             {germanErrorVS20:2f}\t{tttErrorVS20:2f}")
    print(f"Validacion simple (Laplace)   {germanErrorCLPVS20:2f}\t{tttErrorCLPVS20:2f}\n")
    print("Test 12.5%                      German\tTic-Tac-Toe")
    print(f"Validacion cruzada            {germanErrorVC12_5:2f}\t{tttErrorVC12_5:2f}")
    print(f"Validacion cruzada (Laplace)  {germanErrorCLPVC12_5:2f}\t{tttErrorCLPVC12_5:2f}")
    print(f"Validacion simple             {germanErrorVS12_5:2f}\t{tttErrorVS12_5:2f}")
    print(f"Validacion simple (Laplace)   {germanErrorCLPVS12_5:2f}\t{tttErrorCLPVS12_5:2f}\n")
    print("Test 10%                      German\tTic-Tac-Toe")
    print(f"Validacion cruzada            {germanErrorVC10:2f}\t{tttErrorVC10:2f}")
    print(f"Validacion cruzada (Laplace)  {germanErrorCLPVC10:2f}\t{tttErrorCLPVC10:2f}")
    print(f"Validacion simple             {germanErrorVS10:2f}\t{tttErrorVS10:2f}")
    print(f"Validacion simple (Laplace)   {germanErrorCLPVS10:2f}\t{tttErrorCLPVS10:2f}\n")


                               German		Tic-Tac-Toe
Validacion cruzada            0.10680000000000002	0.12692354825720836
Validacion cruzada (Laplace)  0.10680000000000002	0.12692354825720836
Validacion simple             0.1032	0.12385987640272403
Validacion simple (Laplace)   0.1036	0.12385987640272403


# 1. Naive Bayes

## 1.1 Tablas para los distintos porcentajes de Test
El contenido de las tablas es el error calculado al clasificar. Los valores estan sobre 100% por lo que 0.1 significa que ha dado error en el 0.1% del test. 

In [4]:
print_test()

NameError: name 'print_test' is not defined